In [2]:
import pandas as pd

#시군구코드 및 법정동코드 조회
codes_df = pd.read_csv("codes.csv", encoding='cp949')
codes_df = codes_df[codes_df["시군구코드"]!=0]
codes_df = codes_df[codes_df["법정동코드"]!=0]
codes_df['시군구코드'] = codes_df["시군구코드"].astype(str)
codes_df['법정동코드'] = codes_df["법정동코드"].astype(str)
codes_df['시군구동코드'] = codes_df["시군구코드"] + codes_df["법정동코드"]
codes_df.drop_duplicates(subset ="시군구동코드", keep = 'first', inplace = True)
codes_df = codes_df[codes_df['시군구동코드'].map(len) == 10]
codes_df

,시군구코드,법정동코드,행정동코드,시도명,시군구명,법정동명,행정동명,적용시작일,적용만료일,시군구동코드
0,29155,10800,705,광주광역시,남구,진월동,진월동,20210726,99991231,2915510800
1,41390,13500,597,경기도,시흥시,배곧동,배곧2동,20210705,99991231,4139013500
4,41480,38000,390,경기도,파주시,군내면,장단면,20210701,99991231,4148038000
5,41480,38021,390,경기도,파주시,군내면 백연리,장단면,20210701,99991231,4148038021
6,41410,10300,630,경기도,군포시,부곡동,송부동,20210701,99991231,4141010300
...,...,...,...,...,...,...,...,...,...,...
93135,24170,90500,0,광주직할시,북구,동운2동,NaN,19880423,19880423,2417090500
93139,24170,90600,0,광주직할시,북구,풍향1동,NaN,19880423,19880423,2417090600
93170,43130,90200,0,충청북도,충주시,교현2동,NaN,19880423,19880423,4313090200
93178,43130,90100,0,충청북도,충주시,교현1동,NaN,19880423,19880423,4313090100


In [3]:
# HTML 태그 떼는 함수

import re
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [336]:
#""건축물대장 표제부""" 조회 오퍼레이션

from urllib.parse import *
import requests
import pandas as pd
from bs4 import BeautifulSoup


filtered_codes = codes_df[(codes_df['시도명']=="서울특별시") & (codes_df['시군구명']=="강남구")]
print(len(filtered_codes))

variables = ["순번", "대지위치", "시군구코드", "법정동코드", "대지구분코드", "번", "지", "관리건축물대장PK", "대장구분코드", "대장구분코드명", "대장종류코드", "대장종류코드명", "도로명대지위치", "건물명", "특수지명", "블록", "로트", "외필지수", "새주소도로코드", "새주소법정동코드", "새주소지상지하코드", "새주소본번", "새주소부번", "동명칭", "주부속구분코드", "주부속구분코드명", "대지면적(㎡)", "건축면적(㎡)", "건폐율(%)", "연면적(㎡)", "용적률산정연면적(㎡)", "용적률(%)", "구조코드", "구조코드명", "기타구조", "주용도코드", "주용도코드명", "기타용도", "지붕코드", "지붕코드명", "기타지붕", "세대수(세대)", "가구수(가구)", "높이(m)", "지상층수", "지하층수", "승용승강기수", "비상용승강기수", "부속건축물수", "부속건축물면적(㎡)", "총동연면적(㎡)", "옥내기계식대수(대)", "옥내기계식면적(㎡)", "옥외기계식대수(대)", "옥외기계식면적(㎡)", "옥내자주식대수(대)", "옥내자주식면적(㎡)", "옥외자주식대수(대)", "옥외자주식면적(㎡)", "허가일", "착공일", "사용승인일", "허가번호년", "허가번호기관코드", "허가번호기관코드명", "허가번호구분코드", "허가번호구분코드명", "호수(호)", "에너지효율등급", "에너지절감율", "EPI점수", "친환경건축물등급", "친환경건축물인증점수", "지능형건축물등급", "지능형건축물인증점수", "생성일자", "내진 설계 적용 여부", "내진 능력"]
parameter = ["rnum", "platPlc", "sigunguCd", "bjdongCd", "platGbCd", "bun", "ji", "mgmBldrgstPk", "regstrGbCd", "regstrGbCdNm", "regstrKindCd", "regstrKindCdNm", "newPlatPlc", "bldNm", "splotNm", "block", "lot", "bylotCnt", "naRoadCd", "naBjdongCd", "naUgrndCd", "naMainBun", "naSubBun", "dongNm", "mainAtchGbCd", "mainAtchGbCdNm", "platArea", "archArea", "bcRat", "totArea", "vlRatEstmTotArea", "vlRat", "strctCd", "strctCdNm", "etcStrct", "mainPurpsCd", "mainPurpsCdNm", "etcPurps", "roofCd", "roofCdNm", "etcRoof", "hhldCnt", "fmlyCnt", "heit", "grndFlrCnt", "ugrndFlrCnt", "rideUseElvtCnt", "emgenUseElvtCnt", "atchBldCnt", "atchBldArea", "totDongTotArea", "indrMechUtcnt", "indrMechArea", "oudrMechUtcnt", "oudrMechArea", "indrAutoUtcnt", "indrAutoArea", "oudrAutoUtcnt", "oudrAutoArea", "pmsDay", "stcnsDay", "useAprDay", "pmsnoYear", "pmsnoKikCd", "pmsnoKikCdNm", "pmsnoGbCd", "pmsnoGbCdNm", "hoCnt", "engrGrade", "engrRat", "engrEpi", "gnBldGrade", "gnBldCert", "itgBldGrade", "itgBldCert", "crtnDay", "rserthqkDsgnApplyYn", "rserthqkAblty"]
df = pd.DataFrame(columns = variables)
zipped = list(zip(variables, parameter))
url = 'http://apis.data.go.kr/1613000/BldRgstService_v2/getBrTitleInfo'

for i in filtered_codes['시군구동코드']:    
    queryParams = '?' + urlencode({ 
            quote_plus('ServiceKey') : "ZW7sQn/lro4dNI4Glzow/3oLOfWii0CB8RaskqxODpYUq9ZJxs4Ujqp/GyJv7OfIWhf5dkiGBhA3oWgZVSTdPg==", 
            quote_plus('sigunguCd') : i[0:5], 
            quote_plus('bjdongCd') : i[5:10], 
            quote_plus('platGbCd') : '0', 
            quote_plus('bun') : '', 
            quote_plus('ji') : '', 
            quote_plus('startDate') : '', 
            quote_plus('endDate') : '', 
            quote_plus('numOfRows') : '99999', 
            quote_plus('pageNo') : '1' 
            })
    #시군구/동 별로 데이터 불러오기
    result = requests.get(url + queryParams)
    # Parsing
    xmlsoup = BeautifulSoup(result.text, "lxml-xml")
    # Filtering
    te = xmlsoup.findAll("item")

    for t in te:
        data = {}
        for v in zipped:
            data[v[0]]=cleanhtml(str(t.find(v[1])))
        s = pd.Series(data)
        df = df.append(s, ignore_index=True)


35


In [32]:
df = pd.read_csv('C:/Users/skimf/Projects/realEstate/database/11680_buildings_trimmed.csv', encoding='utf-8-sig', low_memory=False)
df['사용승인일'] = df['사용승인일'].replace([' '], 'N')
df

,Unnamed: 0,순번,대지위치,대지구분코드,사용승인일
0,0,1,서울특별시 강남구 압구정동 26-3번지,0,19740313
1,1,2,서울특별시 강남구 압구정동 26-3번지,0,19740313
2,2,3,서울특별시 강남구 압구정동 26-3번지,0,19740313
3,3,4,서울특별시 강남구 압구정동 188-11번지,0,N
4,4,5,서울특별시 강남구 압구정동 188-13번지,0,N
...,...,...,...,...,...
24033,24033,260,서울특별시 강남구 율현동 513번지,0,20150923
24034,24034,261,서울특별시 강남구 율현동 513번지,0,20150923
24035,24035,262,서울특별시 강남구 율현동 513번지,0,20150923
24036,24036,263,서울특별시 강남구 율현동 531번지,0,20160620


In [28]:

df.to_csv("gangnam_buildings_trimmed.csv", encoding='utf-8-sig')

In [ ]:
#데이터 필터링하기
df[df["사용승인일"]<=19500101]["사용승인일"]